# Model Comparison

1. Logistic Regression
2. Random Forest
3. XGBoost

In [7]:
from dotenv import load_dotenv
import sys
import warnings

from __init__ import get_base_path
import data.load_data as load_data
from data.signal import Signal
import models.models as models

from sklearn.linear_model import LogisticRegression

import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

load_dotenv()
warnings.filterwarnings('ignore')

In [8]:
random_seed = int(os.getenv('RANDOM_SEED'))
random_seed

42

In [9]:
csv_file = load_data.combined_csv

if os.path.exists(csv_file):
    pass
else:
    load_data.__main__()
df = pd.read_csv(csv_file,index_col=0)

pd.set_option('display.max_columns', df.shape[1])
pd.set_option('display.max_rows', df.shape[0])

df.head()

,R1_voltage_AC_phase_A_angle,R1_voltage_AC_phase_A_magnitude,R1_voltage_AC_phase_B_angle,R1_voltage_AC_phase_B_magnitude,R1_voltage_AC_phase_C_angle,R1_voltage_AC_phase_C_magnitude,R1_current_AC_phase_A_angle,R1_current_AC_phase_A_magnitude,R1_current_AC_phase_B_angle,R1_current_AC_phase_B_magnitude,R1_current_AC_phase_C_angle,R1_current_AC_phase_C_magnitude,R1_pos_neg_zero_voltage_phase_A_angle,R1_pos_neg_zero_voltage_phase_A_magnitude,R1_pos_neg_zero_voltage_phase_B_angle,R1_pos_neg_zero_voltage_phase_B_magnitude,R1_pos_neg_zero_voltage_phase_C_angle,R1_pos_neg_zero_voltage_phase_C_magnitude,R1_pos_neg_zero_current_phase_A_angle,R1_pos_neg_zero_current_phase_A_magnitude,R1_pos_neg_zero_current_phase_B_angle,R1_pos_neg_zero_current_phase_B_magnitude,R1_pos_neg_zero_current_phase_C_angle,R1_pos_neg_zero_current_phase_C_magnitude,R1_frequency_for_relays,R1_frequency_delta_dFdt_for_relays,R1_appearance_impedance_for_relays,R1_appearance_impedance_angle_for_relays,R1_status_flag_for_relays,R2_voltage_AC_phase_A_angle,R2_voltage_AC_phase_A_magnitude,R2_voltage_AC_phase_B_angle,R2_voltage_AC_phase_B_magnitude,R2_voltage_AC_phase_C_angle,R2_voltage_AC_phase_C_magnitude,R2_current_AC_phase_A_angle,R2_current_AC_phase_A_magnitude,R2_current_AC_phase_B_angle,R2_current_AC_phase_B_magnitude,R2_current_AC_phase_C_angle,R2_current_AC_phase_C_magnitude,R2_pos_neg_zero_voltage_phase_A_angle,R2_pos_neg_zero_voltage_phase_A_magnitude,R2_pos_neg_zero_voltage_phase_B_angle,R2_pos_neg_zero_voltage_phase_B_magnitude,R2_pos_neg_zero_voltage_phase_C_angle,R2_pos_neg_zero_voltage_phase_C_magnitude,R2_pos_neg_zero_current_phase_A_angle,R2_pos_neg_zero_current_phase_A_magnitude,R2_pos_neg_zero_current_phase_B_angle,R2_pos_neg_zero_current_phase_B_magnitude,R2_pos_neg_zero_current_phase_C_angle,R2_pos_neg_zero_current_phase_C_magnitude,R2_frequency_for_relays,R2_frequency_delta_dFdt_for_relays,R2_appearance_impedance_for_relays,R2_appearance_impedance_angle_for_relays,R2_status_flag_for_relays,R3_voltage_AC_phase_A_angle,R3_voltage_AC_phase_A_magnitude,R3_voltage_AC_phase_B_angle,R3_voltage_AC_phase_B_magnitude,R3_voltage_AC_phase_C_angle,R3_voltage_AC_phase_C_magnitude,R3_current_AC_phase_A_angle,R3_current_AC_phase_A_magnitude,R3_current_AC_phase_B_angle,R3_current_AC_phase_B_magnitude,R3_current_AC_phase_C_angle,R3_current_AC_phase_C_magnitude,R3_pos_neg_zero_voltage_phase_A_angle,R3_pos_neg_zero_voltage_phase_A_magnitude,R3_pos_neg_zero_voltage_phase_B_angle,R3_pos_neg_zero_voltage_phase_B_magnitude,R3_pos_neg_zero_voltage_phase_C_angle,R3_pos_neg_zero_voltage_phase_C_magnitude,R3_pos_neg_zero_current_phase_A_angle,R3_pos_neg_zero_current_phase_A_magnitude,R3_pos_neg_zero_current_phase_B_angle,R3_pos_neg_zero_current_phase_B_magnitude,R3_pos_neg_zero_current_phase_C_angle,R3_pos_neg_zero_current_phase_C_magnitude,R3_frequency_for_relays,R3_frequency_delta_dFdt_for_relays,R3_appearance_impedance_for_relays,R3_appearance_impedance_angle_for_relays,R3_status_flag_for_relays,R4_voltage_AC_phase_A_angle,R4_voltage_AC_phase_A_magnitude,R4_voltage_AC_phase_B_angle,R4_voltage_AC_phase_B_magnitude,R4_voltage_AC_phase_C_angle,R4_voltage_AC_phase_C_magnitude,R4_current_AC_phase_A_angle,R4_current_AC_phase_A_magnitude,R4_current_AC_phase_B_angle,R4_current_AC_phase_B_magnitude,R4_current_AC_phase_C_angle,R4_current_AC_phase_C_magnitude,R4_pos_neg_zero_voltage_phase_A_angle,R4_pos_neg_zero_voltage_phase_A_magnitude,R4_pos_neg_zero_voltage_phase_B_angle,R4_pos_neg_zero_voltage_phase_B_magnitude,R4_pos_neg_zero_voltage_phase_C_angle,R4_pos_neg_zero_voltage_phase_C_magnitude,R4_pos_neg_zero_current_phase_A_angle,R4_pos_neg_zero_current_phase_A_magnitude,R4_pos_neg_zero_current_phase_B_angle,R4_pos_neg_zero_current_phase_B_magnitude,R4_pos_neg_zero_current_phase_C_angle,R4_pos_neg_zero_current_phase_C_magnitude,R4_frequency_for_relays,R4_frequency_delta_dFdt_for_relays,R4_appearance_impedance_for_relays,R4_appearance_impedance_angle_for_relays,R4_status_flag_for_re

# Pre-Processing

In [10]:
dp = load_data.DataPreprocessor(df)
df = dp.cast_data_types().get_dataframe()

In [11]:
s = Signal(df)
s.make_waves_iter(relays=('R1','R2','R3','R4'), phases=('A','B','C'), stats=['voltage','current','power'], zero=True)
df = s.data

# Initial Feature Selection

* remove frequency columns because they are basically constant
* remove ids/labels
* remove status flags and logs - sparse data. if needed, can try adding them in later
* we'll first try the waveform instead of individual angle/magnitude features
* we'll first try instant power since it is the product of voltage, current, and impedance, and ignore average power

In [12]:
features = df.copy()
features.drop(columns=dp.status_cols + dp.id_cols + dp.frequencies + dp.magnitudes + dp.angles + dp.impedance, inplace=True)
features.drop(columns=[i for i in features.columns if 'voltage' in i.lower()] + [i for i in features.columns if 'current' in i.lower()] + [i for i in features.columns if 'avg_power' in i], inplace=True)
features.dtypes

marker                                 category
scenario_class                           object
scenario_broad_type                      object
is_attack                                  bool
synthetic_datetime               datetime64[ns]
R1_Phase_A_power                        float64
R1_pos_neg_zero_phase_A_power           float64
R1_Phase_B_power                        float64
R1_pos_neg_zero_phase_B_power           float64
R1_Phase_C_power                        float64
R1_pos_neg_zero_phase_C_power           float64
R2_Phase_A_power                        float64
R2_pos_neg_zero_phase_A_power           float64
R2_Phase_B_power                        float64
R2_pos_neg_zero_phase_B_power           float64
R2_Phase_C_power                        float64
R2_pos_neg_zero_phase_C_power           float64
R3_Phase_A_power                        float64
R3_pos_neg_zero_phase_A_power           float64
R3_Phase_B_power                        float64
R3_pos_neg_zero_phase_B_power           

# Logistic Regression